<a href="https://colab.research.google.com/github/toyinjoy/TabDDM/blob/main/JOY_Vigitel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [9]:
import pandas as pd
import pickle
from tqdm import tqdm
import numpy as np
import scipy

## helpers

### loaders

In [10]:
def load_samples(dataset_dir, dataset):
    with open(f"{dataset_dir}{dataset}/samples.pkl", "rb") as samples_file:
        return pickle.load(samples_file)

In [11]:
def load_split_ids(dataset_dir, dataset, fold_idx, split):
    with open(f"{dataset_dir}{dataset}/fold_{fold_idx}/{split}.pkl", "rb") as ids_file:
        return set(pickle.load(ids_file))

In [12]:
def load_split_samples(dataset_dir, dataset, fold_idx, split):
    split_ids = load_split_ids(dataset_dir, dataset, fold_idx, split)
    with open(f"{dataset_dir}{dataset}/samples.pkl", "rb") as samples_file:
        samples = pickle.load(samples_file)
        return [sample for sample in samples if sample["idx"] in split_ids]

# play

In [13]:
!unzip /content/Vigitel_Dataset.zip

Archive:  /content/Vigitel_Dataset.zip
   creating: Vigitel_Dataset/
   creating: Vigitel_Dataset/fold_0/
  inflating: Vigitel_Dataset/fold_0/test.pkl  
  inflating: Vigitel_Dataset/fold_0/train.pkl  
  inflating: Vigitel_Dataset/fold_0/val.pkl  
   creating: Vigitel_Dataset/fold_1/
  inflating: Vigitel_Dataset/fold_1/test.pkl  
  inflating: Vigitel_Dataset/fold_1/train.pkl  
  inflating: Vigitel_Dataset/fold_1/val.pkl  
   creating: Vigitel_Dataset/fold_2/
  inflating: Vigitel_Dataset/fold_2/test.pkl  
  inflating: Vigitel_Dataset/fold_2/train.pkl  
  inflating: Vigitel_Dataset/fold_2/val.pkl  
   creating: Vigitel_Dataset/fold_3/
  inflating: Vigitel_Dataset/fold_3/test.pkl  
  inflating: Vigitel_Dataset/fold_3/train.pkl  
  inflating: Vigitel_Dataset/fold_3/val.pkl  
   creating: Vigitel_Dataset/fold_4/
  inflating: Vigitel_Dataset/fold_4/test.pkl  
  inflating: Vigitel_Dataset/fold_4/train.pkl  
  inflating: Vigitel_Dataset/fold_4/val.pkl  
  inflating: Vigitel_Dataset/samples.pkl 

In [14]:
dataset = "Vigitel_Dataset"
dataset_dir = "/content/"
folds = [0,1,2,3,4]
num_epochs = 16
num_epochs = 512

In [15]:
selected_features = [
    "cluster_hier_str_setor_num",
    "SANEAMENTO", "SOCIO_ECON", "alcabu", "q6", "civil", "q35", "q47", "q70", "adultos", "af", "q7",
    "hortareg", "sucodia", "flvreco", "atiocu", "freq", "cruadia", "atidom",
    "frutareg", "diab", "atitrans", "tv_d_3", "hart", "saruim",
    "cozidadi_cozidadia", "sofrutad_sofrutadia", "q8_anos"
]
target_features = ["fumante"]
len(selected_features + target_features)

29

In [16]:
samples_df = pd.DataFrame(
    load_samples(dataset_dir, dataset)
)
clusteres_map = samples_df.set_index("cluster_hier_str_setor_num")["cluster_hier_str_setor_label"].to_dict()
samples_df = samples_df[["idx"] + selected_features + target_features]
samples_df.head()

,idx,cluster_hier_str_setor_num,SANEAMENTO,SOCIO_ECON,alcabu,q6,civil,q35,q47,q70,...,frutareg,diab,atitrans,tv_d_3,hart,saruim,cozidadi_cozidadia,sofrutad_sofrutadia,q8_anos,fumante
0,0,3,0.000000,0.362176,0,45,2,2,1,1,...,1,0,0,0,0,0,1,1,11.0,0
1,1,3,0.000000,0.362176,0,45,4,2,1,2,...,1,0,1,0,1,0,1,2,11.0,0
2,2,3,0.000000,0.362176,0,29,1,1,1,2,...,1,0,0,0,0,0,2,1,16.0,0
3,3,3,0.000000,0.328729,0,56,2,1,1,2,...,0,0,0,0,0,0,1,1,20.0,0
4,4,3,0.000000,0.328729,0,30,1,2,1,2,...,0,0,1,1,0,0,2,1,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21759,21759,6,0.169355,0.461191,1,37,2,1,1,2,...,1,0,0,0,0,0,2,1,11.0,1
21760,21760,6,0.169355,0.461191,0,48,1,2,1,2,...,1,0,0,1,0,0,1,1,7.0,0
21761,21761,6,0.169355,0.461191,0,72,3,2,2,2,...,1,0,0,0,1,0,2,1,4.0,0
21762,21762,6,0.169355,0.461191,0,43,2,2,1,2,...,1,0,0,1,0,0,1,2,8.0,0


In [17]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.7 MB/s eta 0:00:00


In [18]:
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframe(
    data = samples_df ,
    table_name='samples_df ')
data = samples_df

In [19]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(data)

synthetic_data = synthesizer.sample(num_rows=10)

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [20]:
synthesizer = GaussianCopulaSynthesizer(
    metadata, # required
    enforce_min_max_values=True,
    enforce_rounding=False,
    #numerical_distributions={
    #    'amenities_fee': 'beta',
    #    'checkin_date': 'uniform'},
    default_distribution='norm'
)

In [21]:
synthesizer.get_parameters()

{'enforce_min_max_values': True,
 'enforce_rounding': False,
 'locales': ['en_US'],
 'numerical_distributions': {},
 'default_distribution': 'norm'}

In [22]:
metadata = synthesizer.get_metadata()
metadata

{
    "tables": {
        "samples_df ": {
            "columns": {
                "idx": {
                    "sdtype": "id"
                },
                "cluster_hier_str_setor_num": {
                    "sdtype": "categorical"
                },
                "SANEAMENTO": {
                    "sdtype": "numerical"
                },
                "SOCIO_ECON": {
                    "sdtype": "numerical"
                },
                "alcabu": {
                    "sdtype": "categorical"
                },
                "q6": {
                    "sdtype": "numerical"
                },
                "civil": {
                    "sdtype": "categorical"
                },
                "q35": {
                    "sdtype": "categorical"
                },
                "q47": {
                    "sdtype": "categorical"
                },
                "q70": {
                    "sdtype": "categorical"
                },
                "adultos":

In [23]:
synthesizer.fit(data)

In [24]:
synthesizer.get_learned_distributions()

{'cluster_hier_str_setor_num': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.5000540572365885,
   'scale': 0.2888067092187676}},
 'SANEAMENTO': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.01628007864625069,
   'scale': 0.047295102159238094}},
 'SOCIO_ECON': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.36775583613540713,
   'scale': 0.12093831903430628}},
 'alcabu': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.4997336224811678,
   'scale': 0.28949078370119763}},
 'q6': {'distribution': 'norm',
  'learned_parameters': {'loc': 48.580913435030325,
   'scale': 17.930529358723238}},
 'civil': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.5001142837440162,
   'scale': 0.28829336326117655}},
 'q35': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.5002904408443951,
   'scale': 0.2876969601946772}},
 'q47': {'distribution': 'norm',
  'learned_parameters': {'loc': 0.4988591029587778,
   'scale': 0.28909842149308496}},

In [25]:
synthesizer.save(
    filepath='my_synthesizer.pkl'
)

In [26]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer.load(
    filepath='my_synthesizer.pkl'
)

In [27]:
clusteres_map

{3: '2-MD-0',
 1: '1-BA-0',
 5: '3-EL-0',
 4: '2-MD-1',
 7: '3-EL-2',
 9: '4-ME-1',
 8: '4-ME-0',
 6: '3-EL-1',
 2: '1-BA-1'}

In [28]:
synthetic_data = synthesizer.sample(num_rows=1_000_000)

# save the data as a CSV
synthetic_data.to_csv('synthetic_data.csv', index=False)

In [29]:


file_path = '/content/synthetic_data.csv'
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())


        idx  cluster_hier_str_setor_num  SANEAMENTO  SOCIO_ECON  alcabu  q6  \
0   4505280                           6    0.040208    0.405951       0  52   
1   6530629                           1    0.000000    0.453487       0  25   
2   4208453                           1    0.067724    0.452887       0  52   
3  12864297                           7    0.075677    0.188950       0  64   
4   3403850                           1    0.064879    0.288439       1  51   

   civil  q35  q47  q70  ...  frutareg  diab  atitrans  tv_d_3  hart  saruim  \
0      4    1    1    1  ...         0     0         0       0     0       0   
1      1    2    1    2  ...         0     0         1       0     0       0   
2      2    2    1    2  ...         0     1         0       0     0       0   
3      4    1    1    2  ...         0     0         1       0     0       0   
4      2    1    2    2  ...         0     0         0       0     0       0   

   cozidadi_cozidadia  sofrutad_sofrutadia  

In [31]:
df.tail()

,idx,cluster_hier_str_setor_num,SANEAMENTO,SOCIO_ECON,alcabu,q6,civil,q35,q47,q70,...,frutareg,diab,atitrans,tv_d_3,hart,saruim,cozidadi_cozidadia,sofrutad_sofrutadia,q8_anos,fumante
999995,6241359,3,0.036439,0.437928,0,45,2,2,1,2,...,0,0,0,0,0,0,1,2,3.847298,1
999996,16708268,1,0.045160,0.501519,0,48,1,2,2,2,...,0,1,1,0,0,1,1,1,14.414592,0
999997,6734739,4,0.075300,0.334994,0,54,2,1,2,2,...,1,0,0,1,0,0,2,3,11.875573,0
999998,3981038,4,0.000441,0.475616,0,31,2,2,1,2,...,1,0,0,0,0,0,1,3,16.346183,0
999999,4840085,7,0.000000,0.302664,0,50,888,1,2,2,...,0,0,0,0,0,0,1,0,10.531958,1
